In [ ]:
pip install tika

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install elasticsearch

In [6]:
import tika
tika.initVM()
from tika import parser
parsed = parser.from_file('test-documents/RBA-VA-VASPs.pdf', xmlContent=True)
html_string = parsed["content"]
print(html_string)

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta name="pdf:docinfo:title" content="DRAFT GUIDANCE FOR A RISK-BASED APPROACH TO VIRTUAL ASSETS AND VIRTUAL ASSET SERVICE PROVIDERS" />
<meta name="xmp:CreatorTool" content="Acrobat PDFMaker 19 for Word" />
<meta name="pdf:docinfo:custom:OECDDocumentValidationWordCount" content="34299" />
<meta name="access_permission:can_print_degraded" content="true" />
<meta name="pdf:docinfo:custom:OECDCountry" content="" />
<meta name="dc:format" content="application/pdf; version=1.6" />
<meta name="pdf:docinfo:creator_tool" content="Acrobat PDFMaker 19 for Word" />
<meta name="pdf:docinfo:custom:OECDCommittee" content="" />
FAT" />
<meta name="access_permission:fill_in_form" content="true" />
<meta name="dc:title" content="DRAFT GUIDANCE FOR A RISK-BASED APPROACH TO VIRTUAL ASSETS AND VIRTUAL ASSET SERVICE PROVIDERS" />
<meta name="OECDDocumentCommittee" content="" />
<meta name="OECDDocumentOriginalLanguage" content="English" />
<meta name="p

In [49]:
from bs4 import BeautifulSoup
from bs4 import NavigableString
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from datetime import datetime

es = Elasticsearch()
es.indices.create(index='poc-reghub', ignore=400)

HTML_TEXT_TAGS = [
    "p", "li", "span", "pre", "center", "h1", "h2", "h3", "h4", "h5", "h6", 
    "big", "small", "b", "s", "i", "u", "tt","sup","sub", "ruby", "rb", "rt", "rp",
    "em", "strong", "ins", "del", "blockquote", "q", "cite", "code", "var", "samp"
]

soup = BeautifulSoup(html_string, "html.parser")
title = soup.find('title').text
pages = soup.find_all(class_='page')

def end_node(tag):
    if not tag.text: #if no text return false
        return False
    elif tag.name not in HTML_TEXT_TAGS and len(tag.find_all(text=False)) > 0: #no other tags inside other than text
        return False
    return True #if valid it reaches here

# for each page, index paragraphs, add pdf title and page number to each paragraph document
pdf_id = 1
page_num = 0
docs = []
for page in pages:
    paragraph_num = 0
    paragraphs = page.find_all(end_node, recursive=True)
    for paragraph in paragraphs:
        docs.append({
            "_index": "poc-reghub",
            "_id": f"{pdf_id}_{page_num}_{paragraph_num}",
            "pdf_id": pdf_id,
            "pdf_title":title, 
            "page_num": page_num,
            "paragraph_num": paragraph_num,
            "text": paragraph.text, 
            "timestamp": datetime.now()})
        paragraph_num += 1
    page_num += 1

res = helpers.bulk(es, docs, chunk_size=40946, request_timeout=2000)